In [1]:
import add_packages
import config
from pprint import pprint

from my_langchain import (
    agent_tools, document_loaders, text_splitters, text_embedding_models, vector_stores,
    chat_models, prompts, utils, output_parsers, agents, documents, runnables,
    llms, histories
)

# Quickstart

Build an agent with two tools: one for online searches and one for specific data retrieval from an index.

## Define tools

In [ ]:
# Create the tools needed: Tavily for online search and a retriever for local index.

tool_tavily_search = agent_tools.tavily_search_results()

# Create a retriever over data. 
loader = document_loaders.WebBaseLoader("https://docs.smith.langchain.com/")
document = loader.load()
documents = text_splitters.RecursiveCharacterTextSplitter(
  chunk_size=1000, chunk_overlap=200,
).split_documents(document)

embeddings = text_embedding_models.OpenAIEmbeddings()
vectorstore = vector_stores.chroma.Chroma.from_documents(documents, embeddings)
retriever = vectorstore.as_retriever()
tool_retriever = vector_stores.create_retriever_tool(
  retriever=retriever,
  name="langsmith_search",
  description="Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

# List of tools will use downstream.
tools = [
  tool_tavily_search, 
  tool_retriever,
]

## Create agent

In [ ]:

# Choose LLM guiding agent.
llm = chat_models.chat_openai

# Choose the prompt to guide the agent.
prompt = prompts.hub.pull("hwchase17/openai-functions-agent")

# Initialize the agent with the LLM, prompt, and tools. 
# The agent takes in input and decides on actions. 
# AgentExecutor execute actions for Agent
agent = agents.create_openai_functions_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = agents.AgentExecutor(agent=agent, tools=tools, verbose=True)

## Run agent

In [ ]:

# Run agent on stateless queries.
agent_executor.invoke({"input": "hi"})

## Adding in memory

This agent is stateless, does not remember previous interactions. To give it memory, pass in previous chat_history. It needs to be called chat_history because of the prompt used. If a different prompt is used, the variable name could be changed.

Keep track of messages automatically by wrapping in a RunnableWithMessageHistory. 

In [ ]:
# Chat history is stored in memory using a global Python dictionary.
store = {}

def get_session_history(
  user_id: str, conversation_id: str
) -> histories.BaseChatMessageHistory:
  """
  Callable references a dict to return an instance of ChatMessageHistory. 
  
  The arguments can be specified by passing a configuration to the 
  RunnableWithMessageHistory at runtime. 
  
  The configuration parameters for tracking message histories can be customized 
  by passing a list of ConfigurableFieldSpec objects to the 
  history_factory_config parameter. 
  
  Two parameters used are user_id and conversation_id.
  """
  if (user_id, conversation_id) not in store:
    store[(user_id, conversation_id)] = histories.ChatMessageHistory()
  return store[(user_id, conversation_id)]

agent_with_memory = runnables.RunnableWithMessageHistory(
  agent_executor,
  get_session_history,
  input_messages_key="input",  # latest input message
  history_messages_key="history",  # key to add historical messages to
  history_factory_config=[
    runnables.ConfigurableFieldSpec(
      id="user_id", annotation=str, name="User ID", default="",
      description="Unique identifier for the user.", is_shared=True,
    ),
    runnables.ConfigurableFieldSpec(
      id="conversation_id", annotation=str, name="Conversation ID", default="", 
      description="Unique identifier for the conversation.", is_shared=True,
    ),
  ]
)

print(agent_with_memory.invoke(
    {"input": "Hi, I'm Bob"},
    config={"configurable": {"user_id": "123", "conversation_id": "1"}}
))

print(agent_with_memory.invoke(
    {"input": "What is my name?"},
    config={"configurable": {"user_id": "123", "conversation_id": "1"}}
))

In [ ]:
message_history = histories.ChatMessageHistory()

agent_with_chat_history = runnables.RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

agent_with_chat_history.invoke(
    {"input": "hi! I'm bob"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)

agent_with_chat_history.invoke(
    {"input": "what's my name?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)

# Agent Types


### OpenAI tools

OpenAI models detect function calls and provide input for API calls. Model outputs JSON object with function arguments for more reliable and useful function calls.

OpenAI termed capability to invoke single function as functions, capability to invoke one or more functions as tools.

Using tools allows the model to request multiple functions to be called when needed.

In [ ]:
# Initialize Tools
tool_tavily_search = agent_tools.TavilySearchResults(max_results=1)
tools = [
  tool_tavily_search
]

# Create Agent
prompt = prompts.hub.pull("hwchase17/openai-tools-agent")

# Choose the LLM that will drive the agent
llm = chat_models.chat_openai

# Construct the OpenAI Tools agent
agent = agents.create_openai_tools_agent(llm, tools, prompt)

# Run Agent
# Create an agent executor by passing in the agent and tools
agent_executor = agents.AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "What is LangcChain?"})

### ReAct

Using an agent to implement the [ReAct](https://react-lm.github.io/) logic.

In [ ]:
# Initialize tools
tool_tavily_search = agent_tools.TavilySearchResults(max_results=1)
tools = [
  tool_tavily_search,
]

# Create Agent
prompt = prompts.hub.pull("hwchase17/react")

# Choose the LLM that will drive the agent
llm = chat_models.chat_openai

# Construct the ReAct agent
agent = agents.create_react_agent(llm=llm, tools=tools, prompt=prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = agents.AgentExecutor(agent=agent, tools=tools, verbose=True)

# Run Agent

In [ ]:
agent_executor.invoke({"input": "What is LangChain?"})

In [ ]:
# Using with chat history
prompt = prompts.hub.pull("hwchase17/react-chat")

# Construct the ReAct agent
agent = agents.create_react_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = agents.AgentExecutor(agent=agent, tools=tools, verbose=True)

message_history = histories.ChatMessageHistory()

agent_with_chat_history = runnables.RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

questions = [
  "hi! I'm bob",
  "what's my name?",
  "Is there any actor with the same name as me?",
]

In [ ]:
agent_with_chat_history.invoke(
    {"input": questions[2]},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)

### Self-ask with search

In [ ]:
# Initialize Tools
tool_tavily_answer = agent_tools.TavilyAnswer(max_results=1, name="Intermediate Answer")
tools = [
  tool_tavily_answer
]

# Create Agent
prompt = prompts.hub.pull("hwchase17/self-ask-with-search")

# Choose the LLM that will drive the agent
llm = chat_models.chat_openai

# Construct the Self Ask With Search Agent
agent = agents.create_self_ask_with_search_agent(llm, tools, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = agents.AgentExecutor(agent=agent, tools=tools, verbose=True,
                                      handle_parsing_errors=True)

# Run Agent

In [ ]:
agent_executor.invoke(
    {"input": "What is the hometown of the reigning men's U.S. Open champion?"})

### XML Agent


### JSON Chat Agent


### Structured chat


# How-to


### Custom agent


In [ ]:
# Create agent using OpenAI Tool Calling for reliability.
# Create it without memory, then add memory for conversation.

#* Load LLM
# Load the language model used to control the agent.
llm = chat_models.chat_openai

#* Define Tools
# Function docstring is important.
# Python function to calculate word length.
@agent_tools.tool
def get_word_length(word: str) -> int:
  """Returns the length of a word."""
  return len(word)

tools = [
  get_word_length,
]

#* Create Prompt for OpenAI Function Calling.
# Input variables: 
# - input: user objective, string
# - agent_scratchpad: sequence of previous agent tool invocations and outputs (messages)
prompt = prompts.ChatPromptTemplate.from_messages([
  (
    "system",
    "You are very powerful assistant, but don't know current events."
  ),
  (
    "user",
    "{input}"
  ),
  prompts.MessagesPlaceholder(variable_name="agent_scratchpad"),
])

#* Bind tools to LLM
# How agent knows tools can be used by relying on OpenAI tool calling LLMs. 
# Tools are passed in OpenAI tool format to the model by binding functions to 
# ensure they are passed each time the model is invoked.
llm_with_tools = llm.bind_tools(tools)

# * Create Agent & Adding memory
# Utility functions: 
# - Component for formatting intermediate steps (agent action, tool output
# pairs) to input messages sent to model
# - Component for converting output message into agent action/agent finish.
agent = (
  {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: agents.format_to_openai_tool_messages(
      x["intermediate_steps"]
    ),
  }
  | prompt 
  | llm_with_tools
  | agents.OpenAIToolsAgentOutputParser()
)

agent_executor = agents.AgentExecutor(agent=agent, tools=tools, verbose=True)

message_history = histories.ChatMessageHistory()
agent_with_chat_history = runnables.RunnableWithMessageHistory(
  agent_executor,
  lambda session_id: message_history,
  input_messages_key="input",
  history_messages_key="chat_history",
)

questions = [
  "Hello",
  "My name is Bob",
  "What is my name?",
  "What is the length of word bob?"
]

In [ ]:

agent_with_chat_history.invoke(
    {"input": questions[3]},
    config={"configurable": {"session_id": "<foo>"}},
)

### Returning Structured Output

Agent return a structured output instead of a single string.

Example, agent doing question-answering over sources. Output should include answer and list of sources used.

In [ ]:
from typing import List
from langchain_core.pydantic_v1 import BaseModel, Field
import json

#* Retriever

# Create a retriever over mock data.
loader = document_loaders.TextLoader("../data/state_of_the_union.txt")
document = loader.load()

text_splitter = text_splitters.RecursiveCharacterTextSplitter(
  chunk_size=1000, chunk_overlap=0,
)
docs = text_splitter.split_documents(document)

# add in the fake source information
# Add a “page_chunk” tag to the metadata of each document.
for i, doc in enumerate(docs):
  doc.metadata["page_chunk"] = i
  
vectorstore = vector_stores.chroma.Chroma.from_documents(
  docs, text_embedding_models.OpenAIEmbeddings(), collection_name="state-of-union"
)
retriever = vectorstore.as_retriever()

#* Tools
# Create tools for the agent, specifically one tool to wrap the retriever.
retriever_tool = vector_stores.create_retriever_tool(
  retriever=retriever,
  name="state-of-union-retriever",
  description="Create tools for the agent, specifically one tool to wrap the retriever.",
)

_.

Create custom parsing logic by passing the Response schema to the LLM via functions parameter, similar to passing tools for the agent to use.

When Response function called by LLM, use as signal to return to user. 
When any other function called by LLM, treat as tool invocation.

Parsing logic:
- If no function is called, assume response to user is AgentFinish
- If Response function is called, respond to user with inputs (structured output) 
and return AgentFinish
- If any other function is called, treat as tool invocation and return AgentActionMessageLog

Using AgentActionMessageLog allows to attach a log of messages for future use in passing back to the agent prompt.


In [ ]:

#* Response schema
# Two fields: answer and list of sources.
class Response(BaseModel):
  """Final response to the question being asked"""
  
  answer: str = Field(
    description="The final response to the user"
  )
  sources: List[int] = Field(
      description=("List of page chunks that contain answer to the question. "
                   "Only include a page chunk if it contains relevant information")
  )
  
#* Custom parsing logic
def parse(output):
  # If no function was invoked, return to user
  if "function_call" not in output.additional_kwargs:
    return agents.AgentFinish(
      return_values={"output": output.content}, log=output.content
    )
  
  # Parse out the function call
  function_call = output.additional_kwargs["function_call"]
  name = function_call["name"]
  inputs = json.loads((function_call["arguments"]))
  
  # If the Response function was invoked, return to the user with the function inputs
  if name == "Response":
    return agents.AgentFinish(return_values=inputs, log=str(function_call)) 
  # Return an agent action
  else:
    return agents.AgentActionMessageLog(
      tool=name, tool_input=inputs, log="", message_log=[output]
    )

#* Create Agent
# prompt: placeholders for user's question and agent_scratchpad (intermediate steps)
prompt = prompts.ChatPromptTemplate.from_messages([
  ("system", "You are a helpful assistant"),
  ("user", "{input}"),
  prompts.MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# tools: attach tools and Response format to LLM as functions
llm = chat_models.chat_openai
llm_with_tools = llm.bind_functions([retriever_tool, Response])

agent = (
  {
    "input": lambda x: x["input"],
    # format agent_scratchpad from intermediate steps (AIMessages, FunctionMessages)
    "agent_scratchpad": lambda x: agents.format_to_openai_function_messages(
      x["intermediate_steps"]
    )
  }
  | prompt
  | llm_with_tools
  # custom output parser: parse LLM response
  | parse
)

# AgentExecutor: run agent-tool loop
agent_executor = agents.AgentExecutor(
  tools=[retriever_tool], agent=agent, verbose=True,
)

In [ ]:
#* Run agent
# It responds with a dictionary answer and sources keys
agent_executor.invoke(
  {"input": "what did the president say about ketanji brown jackson"},
  return_only_outputs=True,
)

### Handle parsing errors

Occasionally LLM cannot determine step to take because outputs are not correctly formatted for output parser. Default agent errors easily control functionality with handle_parsing_errors.


### Access intermediate steps


### Cap the max number of iterations


### Timeouts for agents

### Streaming


### Structured Tools


### Running Agent as an Iterator


# Tools

### Toolkits


### Defining Custom Tools


### Tools as OpenAI Functions

# Test

In [ ]:

# Initialize tools
tool_tavily_search = agent_tools.TavilySearchResults(max_results=1)
tools = [
  tool_tavily_search,
]

# Create Agent
prompt = prompts.hub.pull("hwchase17/react-chat")
# prompt = prompts.hub.pull("hwchase17/react")

# Choose the LLM that will drive the agent
llm = chat_models.chat_openai


my_agent = agents.MyAgent(prompt=prompt, tools=tools, agent_type="react", llm=llm)

questions = [
    "Hello",
    "My name is Bob",
    "what's my name?",
    "Is there any actor with the same name as me?",
]

In [ ]:
response = my_agent.invoke_agent(questions[2])